# Set up

In [19]:
import os, zipfile, glob, json, string, shutil

from google.colab.files import upload
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential

import tensorflow.keras.utils as ku
import tensorflow.keras.layers as layers
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Dataset

## Kaggle

In [3]:
print("Installing kaggle")
!pip install kaggle -q
print("Upload kaggle.json")
upload()
print("Setting kaggle up...")
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!rm -fr sample_data
print("Done!")

Installing kaggle
Upload kaggle.json


Saving kaggle.json to kaggle.json
Setting kaggle up...
Done!


## Downloading dataset

In [4]:
data_name = "data"
train_data = "data/train"
test_data = "data/test"

In [5]:
if not os.path.exists(data_name):
  !kaggle datasets download -d taejinwoo/multiwoz-22
  with zipfile.ZipFile("multiwoz-22.zip", "r") as zip_ref: zip_ref.extractall("./")
  os.remove("multiwoz-22.zip")
  os.rename("MultiWOZ_2.2", data_name)

 95% 14.0M/14.8M [00:01<00:00, 15.2MB/s]
100% 14.8M/14.8M [00:01<00:00, 7.81MB/s]


## Dataframe

In [6]:
sentences = []
for i, filepath in enumerate(glob.glob(f"{train_data}/*.json")):
  with open(filepath, 'r') as f:
    data = json.load(f)

  utterances = []
  for item in data:
    for turn in item["turns"]:
      utterances.append(turn["utterance"])

  sentences.append(utterances)

df_ = pd.DataFrame({"sentences": sentences})
print("Total num of rows", len(df_.index))
df_.head()

Total num of rows 17


,sentences
0,[I am looking for the Addenbrookes Hospital wi...
1,[I am looking for something fun to do in the s...
2,[i need a place to dine in the center thats ex...
3,[I need a train to stansted airport that leave...
4,"[Hi, I'm looking for places to visit. Mainly i..."


In [22]:
max_rows = 500
all_sentences = []

for i in range(len(df_.loc[:, "sentences"]) - 1):
  sentences = df_.loc[i, "sentences"]

  for sentence in sentences:
    if len(all_sentences) + 1 > max_rows: break

    all_sentences.append(sentence)

In [23]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_sentences]
corpus[:10]

['i am looking for the addenbrookes hospital with hepatology department',
 'the telephone number is 01223217712',
 'thank you goodbye',
 'have a wonderful day',
 'im planning a trip to cambridge and need a place to dine can you find something that serves jamaican food in the centre',
 'there are no jamaican restaurants in the centre would you like to try another area or another food type',
 'are you sure it should be expensive',
 'im sorry there doesnt seem to be a jamaican restaurant in centre would you like me to look for something else',
 'i really want jamaican food can you check another area if you find one it should be expensive if not ill try thai in the centre',
 'that should be bangkok city its address is 24 green street city centrecb23jx you need the phone number']

## Create n-gram sequences

In [24]:
tokenizer = Tokenizer()

def create_sequences(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1

  sequences = []
  for sentence in corpus:
    tok_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, total_words):
      n_gram_seq = tok_sentence[:i+1]
      sequences.append(n_gram_seq)

  return sequences, total_words

inp_sequences, total_words = create_sequences(corpus)
inp_sequences[:10]

[[3, 52],
 [3, 52, 41],
 [3, 52, 41, 7],
 [3, 52, 41, 7, 2],
 [3, 52, 41, 7, 2, 340],
 [3, 52, 41, 7, 2, 340, 219],
 [3, 52, 41, 7, 2, 340, 219, 23],
 [3, 52, 41, 7, 2, 340, 219, 23, 458],
 [3, 52, 41, 7, 2, 340, 219, 23, 458, 341],
 [3, 52, 41, 7, 2, 340, 219, 23, 458, 341]]

## Get inputs and labels

In [25]:
def split_sequences_labels(sequences, pad_len):
  padded_sequences = np.array(ku.pad_sequences(sequences, maxlen=pad_len, padding="pre"))

  inputs, labels = padded_sequences[:,:-1], padded_sequences[:,-1]
  labels = ku.to_categorical(labels, num_classes=total_words)
  return inputs, labels

max_seq_len = max([len(seq) for seq in inp_sequences])
inputs, labels = split_sequences_labels(inp_sequences, max_seq_len)

# LSTM

## Building model

In [28]:
loss = tf.keras.losses.CategoricalCrossentropy()
optim = tf.keras.optimizers.Adam()

input_len = max_seq_len - 1

model = Sequential()

# Add Input Embedding Layer
model.add(layers.Embedding(256, 100, input_length=input_len))

# Add Dense Layers
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))

# Add LSTM Layer
model.add(layers.LSTM(512))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(total_words, activation='softmax'))

model.compile(loss=loss, optimizer=optim)

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 45, 100)           25600     
                                                                 
 dense_40 (Dense)            (None, 45, 1024)          103424    
                                                                 
 dense_41 (Dense)            (None, 45, 512)           524800    
                                                                 
 dense_42 (Dense)            (None, 45, 256)           131328    
                                                                 
 dense_43 (Dense)            (None, 45, 128)           32896     
                                                                 
 dense_44 (Dense)            (None, 45, 64)            8256      
                                                                 
 dense_45 (Dense)            (None, 45, 32)          

## Train

In [29]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]


epochs = 20
batch_size = 64
history = model.fit(inputs, labels, epochs=epochs, batch_size=batch_size, callbacks=callbacks)

Epoch 1/20
6696/6696 [==============================] - 245s 35ms/step - loss: 0.4679
Epoch 2/20
6696/6696 [==============================] - 224s 33ms/step - loss: 0.1041
Epoch 3/20
6696/6696 [==============================] - 221s 33ms/step - loss: 0.0866
Epoch 4/20
6696/6696 [==============================] - 222s 33ms/step - loss: 0.0763
Epoch 5/20
6696/6696 [==============================] - 221s 33ms/step - loss: 0.0705
Epoch 6/20
6696/6696 [==============================] - 220s 33ms/step - loss: 0.0659
Epoch 7/20
6696/6696 [==============================] - 221s 33ms/step - loss: 0.0625
Epoch 8/20
6696/6696 [==============================] - 221s 33ms/step - loss: 0.0592
Epoch 9/20
6696/6696 [==============================] - 218s 32ms/step - loss: 0.0551
Epoch 10/20
6696/6696 [==============================] - 216s 32ms/step - loss: 0.0549
Epoch 11/20
6696/6696 [==============================] - 215s 32ms/step - loss: 0.0497
Epoch 12/20
6696/6696 [=============================

KeyboardInterrupt: ignored

In [30]:
model.save("sentence_completion_tf_model")
shutil.make_archive("sentence_completion_tf_model", "zip", "sentence_completion_tf_model")

'/content/sentence_completion_tf_model.zip'

## Inference

In [31]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = ku.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word.lower()
                break
        seed_text += " "+output_word

    return seed_text.title()

In [36]:
print(generate_text("Hi", 10 , model, max_seq_len))
print(generate_text("Today I was going to", 7, model, max_seq_len))
print(generate_text("My address is", 20, model, max_seq_len))
print(generate_text("I", 7, model, max_seq_len))
print(generate_text("Would you", 20, model, max_seq_len))

Hi Is Like Address Is Between Victoria Road And The River
Today I Was Going To Cambridge The Hotel To The Restaurant Please
My Address Is The The Nightclub Priced Is Cb11Jg Is The Anything Else I Can Help You With Today Today Today Today Is
I Am Looking For Information In Cambridge Centre
Would You Goodbye Me Find A Restaurant In The South Centre Of You Like A Room Number Please Cambridge A The 4
